In [1]:
import glob
import sys
import math
import numpy as np
import tensorflow as tf
from random import shuffle
from random import seed

from device import Device
from stroke import Stroke
from sample import Sample
from datetime import datetime
import pickle
import sklearn.utils
import statistics
import tflearn

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
class MySample:
    def __init__(self, events, sequence_counter, count, inch, jump):
        self.angle = 0
        self.inch = inch

        self.x = []
        self.y = []
        self.time = []
        for i in range(sequence_counter, sequence_counter+count):
            if i >= jump-1:
                self.x.append(events[jump-1][0])
                self.y.append(events[jump-1][1])
                
                if i == sequence_counter:
                    self.time.append(17)
                else:
                    self.time.append(events[jump-1][3]-events[jump-2][3])
            else:
                self.x.append(events[i][0])
                self.y.append(events[i][1])
                if i == sequence_counter:
                    self.time.append(17)
                else:
                    self.time.append(events[i][3]-events[i-1][3])

    def derivate(self):
        x_der = []
        y_der = []
        time_der = []
        for i in range(len(self.x)-1):
            x_der.append(self.x[i+1]-self.x[i])
            y_der.append(self.y[i+1]-self.y[i])
            time_der.append(self.time[i+1])
        self.x = x_der
        self.y = y_der
        self.time = time_der

    def getAngle(self, startIndex, endIndex):
        myX = self.x[endIndex]-self.x[startIndex]
        myY = self.y[endIndex]-self.y[startIndex]
        return math.atan2(myX, myY)

    def rotate(self, angle, originIndex):
        self.rotation = angle

        cs = math.cos(angle)
        sn = math.sin(angle)
        for i in range(len(self.x)):
            myX = (self.x[i]-self.x[originIndex]) * cs -\
                (self.y[i]-self.y[originIndex])*sn
            myY = (self.x[i]-self.x[originIndex]) * sn +\
                (self.y[i]-self.y[originIndex])*cs
            self.x[i] = myX + self.x[originIndex]
            self.y[i] = myY + self.y[originIndex]

In [3]:
def loadStudyData(path, inType, sampleLength):
    file_list = []
    name = "xxx"
    if inType == 1: name = "FittsTasks-participant"
    elif inType == 2: name = "PaintTasks-participant"
    elif inType == 3: name = "WriteTasks-participant"

    fileName = path+name

    for index in range(1,9):
        file_list.append(fileName+str(index)+".txt")

    samples = []

    for fileName in file_list:
        events = []
        f = open(fileName, 'r')
        for line in f:
            tokens = line.split(';')
            events.append([float(tokens[2]), float(tokens[3]), float(tokens[4]), int(tokens[1]), int(tokens[5])])

        seqCount = 0
        for i, event in enumerate(events):
            jump = i+sampleLength+1

            for j in range(i, i+sampleLength):
                if j >= len(events) or  events[j][4] is not 2:
                    jump = j
                    break
            if jump-i > 11:
                sample = MySample(events, i, sampleLength, 7, jump)

                sample.angle = sample.getAngle(9, 10) + math.radians(45)
                sample.rotate(sample.angle, 10)

                sample.derivate()

                sameTime = 0
                for a in range(len(sample.time)):
                    if sample.time[a] < 1:
                        sameTime = 1
                if sameTime == 0:
                    samples.append(sample)

    return samples

In [4]:
def buildLSTM():
    net = tflearn.input_data([None, 10, 3], name="input1")
    net = tflearn.lstm(net, 512, return_seq=True, weights_init="xavier")
    net = tflearn.dropout(net, 0.75)
    net = tflearn.lstm(net, 256, weights_init="xavier")
    net = tflearn.dropout(net, 0.75)
    net = tflearn.fully_connected(net, 2, activation='linear', weights_init="xavier")
        
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.0001, loss='mean_square')

    return tflearn.DNN(net)

def buildLSTMVectors(samples, steps):
    inStudyVec = []
    outStudyVec = []
    
    for sample in samples:
        line = []
        for i in range(10):
            line.append([sample.x[i], sample.y[i], sample.time[i+steps]])
        inStudyVec.append(line)
        
        x = 0
        y = 0
        for i in range(0,steps):
            x = x + sample.x[10 + i]
            y = y + sample.y[10 + i]
        outStudyVec.append([x, y])
    inStudyVec = np.array(inStudyVec)
    outStudyVec = np.array(outStudyVec)

    return inStudyVec, outStudyVec

def getLSTMPerformance(model, inVec, outVec, steps):
    batch_size = 500
    total_batch = int(len(inVec)/batch_size)
    avgDist = 0
    for i in range(int(len(inVec)/batch_size)+1):
        batch_x = inVec[i*batch_size:min((i+1)*batch_size, len(inVec))]
        batch_y = outVec[i*batch_size:min((i+1)*batch_size, len(inVec))]

        myY = np.array(model.predict(batch_x))

        for j in range(len(batch_x)):           
            dist = (batch_y[j][0]-myY[j][0])*(batch_y[j][0]-myY[j][0])
            dist = dist + (batch_y[j][1]-myY[j][1])*(batch_y[j][1]-myY[j][1])
            dist = math.sqrt(dist)
            avgDist = avgDist + dist
    return avgDist/len(inVec)

In [5]:
tasks = ["fitts", "draw", "write"]
time = ["LSTM 33", "LSTM 67", "LSTM 100"]
        
model = buildLSTM()
for j1, store in enumerate(time):
    j = (j1+1) * 2
    model.load("./models/"+store+'.tflearn', False)
    avgError = 0
    for i in range(1, 4):
        samples = loadStudyData('./data/', i, 11+j)
        inStudyVec, outStudyVec = buildLSTMVectors(samples, j)
        perf = getLSTMPerformance(model, inStudyVec, outStudyVec, j)
        avgError = avgError + perf
        
        print("{:.2f}".format(j*16.6666), "ms ", tasks[i-1], "{:.1f}".format(perf), "px")
    print("{:.2f}".format(j*16.6666), "ms ", "average", "{:.1f}".format(avgError/3), "px\n")

INFO:tensorflow:Restoring parameters from /home/henzens/jupyter/touch/models/LSTM 33.tflearn
33.33 ms  draw 7.4 px
33.33 ms  write 13.2 px
33.33 ms  fitts 5.1 px
33.33 ms  average 8.6 px

INFO:tensorflow:Restoring parameters from /home/henzens/jupyter/touch/models/LSTM 67.tflearn
66.67 ms  draw 15.6 px
66.67 ms  write 31.9 px
66.67 ms  fitts 13.0 px
66.67 ms  average 20.2 px

INFO:tensorflow:Restoring parameters from /home/henzens/jupyter/touch/models/LSTM 100.tflearn
100.00 ms  draw 25.4 px
100.00 ms  write 54.3 px
100.00 ms  fitts 23.0 px
100.00 ms  average 34.2 px

